**Make plots of monitoring elements**

In particular:
- Limited to making plots of one or multiple 1D MEs.
- Make plots all lumisections in a selected run (maybe extend later).

In [ ]:
### imports

# external modules
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib

# local modules
sys.path.append('../utils')
import dataframe_utils as dfu
import hist_utils as hu
import plot_utils as pu
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(pu)

In [ ]:
### read the data
# OPTION 1: from previously stored files

# define year to use
year = '2024'

# define ME names to use 
menames = ([
    'PixelPhase1-Tracks-PXBarrel-charge_PXLayer_2',
    'PixelPhase1-Tracks-PXBarrel-charge_PXLayer_3',
    'PixelPhase1-Tracks-PXBarrel-charge_PXLayer_4',
    #'PixelPhase1-Tracks-PXForward-charge_PXDisk_-1',
    #'PixelPhase1-Tracks-PXForward-charge_PXDisk_-2',
    #'PixelPhase1-Tracks-PXForward-charge_PXDisk_-3',
    #'PixelPhase1-Tracks-PXForward-charge_PXDisk_+1',
    #'PixelPhase1-Tracks-PXForward-charge_PXDisk_+2',
    #'PixelPhase1-Tracks-PXForward-charge_PXDisk_+3',
])

# define datasets to use
datasets = ([
    'ZeroBias-Run2024B-PromptReco-v1',
    #'ZeroBias-Run2024C-PromptReco-v1',
    #'ZeroBias-Run2024D-PromptReco-v1',
    #'ZeroBias-Run2024E-PromptReco-v1',
])

# initialize dictionary for the dataframes
# (one entry per ME)
dfs = {}

# loop over the ME types to take into account
for mename in menames:
    print('Adding {}...'.format(mename))
    # loop over datasets
    dflist = []
    for dataset in datasets:
        filename = '{}-DQMIO-{}.parquet'.format(dataset, mename)
        datadir = '../../SWAN_projects/ML4DQM-DC/data/dqmio2024_charge'
        filename = os.path.join(datadir, filename)
        df = pd.read_parquet(filename)
        dflist.append(df)
    # concatenate dataframes for datasets
    df = pd.concat(dflist,ignore_index=True)
    dfs[mename] = df
    
# do some printouts
print('Found following histograms:')
for mename in menames:
    print('{}: {} instances'.format(mename, len(dfs[mename])))

In [ ]:
### read the data
# OPTION 2: directly from DIALS

# imports and authentication
from cmsdials.auth.bearer import Credentials
from cmsdials import Dials
from cmsdials.filters import RunFilters
from cmsdials.filters import LumisectionHistogram1DFilters

creds = Credentials.from_creds_file()
dials = Dials(creds, workspace='tracker')

In [ ]:
### read the data
# OPTION2, continued

# find out in which dataset the run of interest is located
dataset = '/ZeroBias/Run.*-PromptReco-v1/DQMIO'
run_number = 381364
filters = RunFilters(
    dataset__regex=dataset,
    run_number=run_number
)

data = dials.run.list_all(
    filters,
    enable_progress=True
)
df = data.to_pandas()
df.head()

In [ ]:
### read the data
# OPTION2, continued

# filter settings
dataset = '/ZeroBias/Run2024E-PromptReco-v1/DQMIO'
menames = ([
    'PixelPhase1/Tracks/PXBarrel/charge_PXLayer_2',
    'PixelPhase1/Tracks/PXBarrel/charge_PXLayer_3',
    'PixelPhase1/Tracks/PXBarrel/charge_PXLayer_4',
    'PixelPhase1/Tracks/PXForward/charge_PXDisk_-1',
    'PixelPhase1/Tracks/PXForward/charge_PXDisk_-2',
    'PixelPhase1/Tracks/PXForward/charge_PXDisk_-3',
    'PixelPhase1/Tracks/PXForward/charge_PXDisk_+1',
    'PixelPhase1/Tracks/PXForward/charge_PXDisk_+2',
    'PixelPhase1/Tracks/PXForward/charge_PXDisk_+3',
])
run_number = 381364

dfs = {}
for mename in menames:
    data = dials.h1d.list_all(
        LumisectionHistogram1DFilters(
            dataset=dataset,
            me=mename,
            run_number=run_number
        ),
        enable_progress=True
    )
    df = data.to_pandas()
    dfs[mename] = df

# do some printouts
print('Found following histograms:')
for mename in menames:
    print('{}: {} instances'.format(mename, len(dfs[mename])))

In [ ]:
### filter the data (optional)

# location of DCS-on json file
# note: make sure to modify or add the correct json files depending on your data
dcsjson = '/eos/user/c/cmsdqm/www/CAF/certification/Collisions24/DCSOnly_JSONS/dailyDCSOnlyJSON/Collisions24_13p6TeV_378981_381594_DCSOnly_TkPx.json'

# filter the data
# note: modify to your taste
dffs = {}
for mename in menames:
    dffs[mename] = dfu.select_json(dfs[mename], dcsjson)

print('Filtered number of runs: {}'.format(len(dfu.get_runs(dffs[menames[0]]))))
print('Filtered number of LS: {}'.format(len(dffs[menames[0]])))
print('All available run numbers in selected data:\n{}'.format(dfu.get_runs(dffs[menames[0]])))

In [ ]:
### settings for plot aesthetics

xaxtitledict = {mename: 'Cluster charge (electrons)' for mename in menames}
yaxtitledict = {mename: 'Number of clusters' for mename in menames}
caxtitle = 'Lumisection number'
extratextdict = {mename: '_'.join(mename.split('_')[-2:]) for mename in menames}
extracmstext = 'Preliminary'
lumitext = '2024 (13.6 TeV)'

In [ ]:
### plot variation over a run

# settings
run = 381364
ncols = 3
nrows = 3

# get the histograms
dfrs = {}
for mename in menames:
    dfrs[mename] = dfu.select_runs(dffs[mename], [run])
        
# loop over all histogram types
fig,axs = plt.subplots(nrows, ncols, figsize=(5*ncols,3.5*nrows), squeeze=False)
for j,mename in enumerate(menames):
    # get the histograms
    ls = np.array(dfrs[mename]['ls_number'].values)
    hists = np.vstack(dfrs[mename]['data'].values)
    # other settings
    xmin = dfrs[mename]['x_min'].values[0]
    xmax = dfrs[mename]['x_max'].values[0]
    # make the plot
    ax = axs[int(j/ncols),j%ncols]
    pu.plot_hists_multi( hists.tolist(),
                                fig=fig, ax=ax,
                                colorlist=ls,
                                xlims=(xmin,xmax),
                                xaxtitle = xaxtitledict[mename], xaxtitlesize=17,
                                yaxtitle = yaxtitledict[mename], yaxtitlesize=17,
                                caxtitle = caxtitle, caxtitlesize=15, caxtitleoffset=15,
                                ymaxfactor = 1,
                                legendsize = 14, ticksize = 14 )
    pu.add_cms_label( ax, pos=(0.05,1.01), extratext=extracmstext, fontsize=16 )
    pu.add_text( ax, 'Run {}'.format(run), (0.97,0.9), fontsize=15, horizontalalignment='right' )
    pu.add_text( ax, extratextdict[mename], (0.97,0.82), fontsize=15, horizontalalignment='right' )
    pu.add_text( ax, lumitext, (0.71,1.01), fontsize=14 )

fig.subplots_adjust(wspace=0.2, hspace=0.3)
fig.tight_layout()
plt.show()        